In [34]:
# import package
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from scipy import stats
import statistics
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity

In [35]:
# load the data
df = pd.read_excel('/Users/dantoni/HW1/TP1.xls', skiprows = 4)

# EXERCICE 1
## EXERCICE 1.1

In [36]:
# Compute Arithmetic return for each stock
AR = pd.DataFrame((df.iloc[1:,1:].values - df.iloc[0:-1,1:].values) / df.iloc[0:-1,1:].values,  columns = ['Microsoft','Credit Suisse' ,'Boeing' ,'Coca Cola' ,'Nike'])

In [37]:
AR['Microsoft'].values/3

array([ 0.00481348, -0.00711744,  0.00181818, ..., -0.0040138 ,
       -0.00533232,  0.00987033])

## EXERCICE 1.2

In [38]:
# Create a function which return the centered moments
def center_mom(x):
    print('Mean: ', statistics.mean(x),
    '\nVariance: ', statistics.variance(x),
    '\nSkewness: ', stats.skew(x),
    '\nKurtosis: ', stats.kurtosis(x, fisher = False))
    return 

# EXERCICE 2

## EXERCICE 2.1

In [39]:
# Create the Gaussian function
def gaussian_fct(x): 
    return 1/math.sqrt(2*math.pi)*math.exp(-(x**2)/2)

f = np.vectorize(gaussian_fct) # Allow an array as argument

In [40]:
# Create the kernel estimation function
def gaussiankernel(obs, nb_points, bandwidth):
    kernel_points = np.linspace(min(obs), max(obs), nb_points)
    result = np.ones(len(kernel_points))
    for x in range(0,len(kernel_points)):
        result[x] = (statistics.mean(f((kernel_points[x] - obs) / bandwidth)) / bandwidth)
    return np.array([kernel_points, result])

In [41]:

pts_2 = np.linspace(0, 3, 3)
nb_points = 3

In [42]:
np.outer(np.ones(nb_points), pts_2)

array([[0. , 1.5, 3. ],
       [0. , 1.5, 3. ],
       [0. , 1.5, 3. ]])

In [43]:
np.concatenate(np.outer(np.ones(nb_points), pts_2), axis=0)

array([0. , 1.5, 3. , 0. , 1.5, 3. , 0. , 1.5, 3. ])

In [44]:
# Plot the kernel estimation of the returns for each company
company = ['Microsoft','Credit Suisse' ,'Boeing' ,'Coca Cola' ,'Nike']
for i in company:
    obs = AR[i].values
    nb_points = 1000
    bandwidth = np.sqrt(statistics.variance(obs))*len(obs)**(-1/5)
    a = gaussiankernel(obs, nb_points, bandwidth)
    fig = go.Figure(data = go.Scatter(x = a[0], y = a[1]))
    fig.update_layout(title = i, xaxis_title = 'Return', yaxis_title = 'Density')
    fig.show()

## EXERCICE 2.2

In [22]:
# Compute the kernel estimation of the returns for Credit Suisse with different bandwidths
obs = AR['Credit Suisse'].values
nb_points = 1000
bandwidth1 = np.sqrt(statistics.variance(obs))*len(obs)**(-1/5)
bandwidth2 = 0.0001
bandwidth3 = 0.001
bandwidth4 = 0.01
a1 = gaussiankernel(obs, nb_points, bandwidth1)
a2 = gaussiankernel(obs, nb_points, bandwidth2)
a3 = gaussiankernel(obs, nb_points, bandwidth3)
a4 = gaussiankernel(obs, nb_points, bandwidth4)

In [23]:
# Plot the kernel estimation of the returns for Credit Suisse with different bandwidths
fig0 = make_subplots(rows=2, cols=2)
fig0.add_trace((go.Scatter(x = a2[0], y = a2[1], name='0.0001')), row=1, col=1)
fig0.add_trace((go.Scatter(x = a3[0], y = a3[1], name='0.001')), row=1, col=2)
fig0.add_trace((go.Scatter(x = a1[0], y = a1[1], name= round(np.sqrt(statistics.variance(obs))*len(obs)**(-1/5), 4))), row=2, col=1)
fig0.add_trace((go.Scatter(x = a4[0], y = a4[1], name='0.01')), row=2, col=2)

fig0.update_xaxes(title_text = "", row=1, col=1)
fig0.update_xaxes(title_text = "", row=1, col=2)
fig0.update_xaxes(title_text = "Return", row=2, col=1)
fig0.update_xaxes(title_text = "Return", row=2, col=2)

fig0.update_yaxes(title_text= "Density", row=1, col=1)
fig0.update_yaxes(title_text= "", row=1, col=2)
fig0.update_yaxes(title_text= "Density", row=2, col=1)
fig0.update_yaxes(title_text= "", row=2, col=2)

fig0.update_layout(title = 'Credit Suisse', legend_title = "Bandwidth")

fig0.show()

## EXERCICE 2.3 (version 2 plots)

In [ ]:
# Create a matrix which receive the new data (column 1 for the first half and column 2 for the seconde half)
number_obs = math.floor(len(AR)/2)
data_divised = np.ones((number_obs,2))

In [ ]:
# Separate data in half
for i in range(0,2):
    data_divised[:, i] = AR['Credit Suisse'][number_obs*i:number_obs*(i+1)]
    
# Compute the kernel estimation of the returns for the first half of Credit Suisse
obs = data_divised[:,0]
nb_points = 1000
bandwidth = np.sqrt(statistics.variance(obs))*len(obs)**(-1/5)
b = gaussiankernel(obs, nb_points, bandwidth)

# Compute the kernel estimation of the returns for the second half of Credit Suisse
obs = data_divised[:,1]
c = gaussiankernel(obs, nb_points, bandwidth)

In [ ]:
# Plot the kernel estimation of the returns for the two half of Credit Suisse (in 2 subplots)
fig2 = make_subplots(rows=1, cols=2)
fig2.add_trace((go.Scatter(x = b[0], y = b[1], name='1st part')), row=1, col=1)
fig2.add_trace((go.Scatter(x = c[0], y = c[1], name='2nd part')), row=1, col=2)
fig2.update_layout(title = 'Credit Suisse', xaxis_title = 'Return', yaxis_title = 'Density')
fig2.update_xaxes(title_text = "Return", row=1, col=2)
fig2.show()

## Exercice 2.3 (version 1 plot)

In [ ]:
# Plot the kernel estimation of the returns for the two half of Credit Suisse (in 1 plot)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=b[0], y=b[1],
                    name='1st part'))
fig1.add_trace(go.Scatter(x=c[0], y=c[1],
                    name='2nd part'))

fig1.update_layout(title = 'Credit Suisse', xaxis_title = 'Return', yaxis_title = 'Density')
fig1.show()

# EXERCICE 3

## EXERCICE 3.1

In [ ]:
# Simulate 1000 observations normaly distributed
simulation = np.random.normal(0, 1, 1000)

## EXERCICE 3.2

In [ ]:
# Compute the kernel estimation of the simulation
obs = simulation
nb_points = 1000
bandwidth = np.sqrt(statistics.variance(obs))*len(obs)**(-1/5)
d = gaussiankernel(obs, nb_points, bandwidth)

# Plot the kernel estimation of the simulation
fig3 = go.Figure(data = go.Scatter(x = d[0], y = d[1]))
fig3.update_layout(title = 'Gaussian Kernel estimation of normal simulation', xaxis_title = 'Observation', yaxis_title = 'Density')
fig3.show()

## EXERCICE 3.3

In [ ]:
# Simulate 1000 observations beta distributed
simulation_beta = np.random.beta(1.5, 4, 1000)

# Compute the kernel estimation of the simulation
obs = simulation_beta
nb_points = 1000
bandwidth = np.sqrt(statistics.variance(obs))*len(obs)**(-1/5)
e = gaussiankernel(obs, nb_points, bandwidth)

# Plot the kernel estimation of the simulation
fig4 = go.Figure(data = go.Scatter(x = e[0], y = e[1]))
fig4.update_layout(title = 'Gaussian Kernel estimation of beta simulation', xaxis_title = 'Observation', yaxis_title = 'Density')
fig4.show()

# EXERCICE 4

## EXERCICE 4.1

In [ ]:
# Compute the kernel estimation of the returns for Credit Suisse
obs = AR['Credit Suisse'].values
nb_points = 1000
bandwidth = np.sqrt(statistics.variance(obs))*len(obs)**(-1/5)
g = gaussiankernel(obs, nb_points, bandwidth)

# Compute the normal pdf parameterized with the returns for Credit Suisse (mean and sigma of the returns)
mu_hat, sigma_hat = stats.norm.fit(obs)
xmin = min(obs)
xmax = max(obs)
x = np.linspace(xmin, xmax, 1000)
p = norm.pdf(x, mu_hat, sigma_hat)

In [ ]:
# Plot the kernel estimation and the normal pdf of the returns for Credit Suisse
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=g[0], y=g[1],
                    name='Real data'))
fig5.add_trace(go.Scatter(x=x, y=p,
                    name='Normal distribution'))

fig5.update_layout(title = 'Credit Suisse', xaxis_title = 'Return', yaxis_title = 'Density')
fig5.show()

## EXERCICE 4.2

In [ ]:
# Create the bivariate kernel estimation function
def bivariatekernel(data1, data2, nb_points, bandwidth):
    kernel_points1 = np.linspace(min(data1), max(data1), nb_points)
    kernel_points2 = np.linspace(min(data2), max(data2), nb_points)
    result = np.ones(len(kernel_points1)**2)
    for yi in range(0,len(kernel_points2)):
        for xi in range(0,len(kernel_points1)):
            result[xi+yi*len(kernel_points1)] = (statistics.mean(f((kernel_points1[xi] - data1) / bandwidth)*f((kernel_points2[yi] - data2) / bandwidth)) / bandwidth**2)
    return np.array([np.tile(kernel_points1, len(kernel_points1)), np.repeat(kernel_points2,len(kernel_points1)), result])

In [ ]:
# Compute the bivariate kernel estimation of the returns for Credit Suisse and Microsoft
data1 = AR['Credit Suisse'].values
data2 = AR['Boeing'].values
nb_points = 50
bandwidth = np.sqrt(statistics.variance(data1))*len(data1)**(-1/5) # à trouver
k = bivariatekernel(data1, data2, nb_points, bandwidth)


In [ ]:
# Plot the bivariate kernel estimation of the returns for Credit Suisse and Microsoft
fig6 = go.Figure(data=[go.Scatter3d(x = k[0], y = k[1], z = k[2], mode = 'lines', marker = dict(size = 1))])
fig6.update_layout(title = 'Joint Density', scene = dict(xaxis = dict(title = 'Credit Suisse'), yaxis = dict(title = 'Boeing'), zaxis = dict( title = 'Density'),))
fig6.show()

## EXERCICE 4.3

In [ ]:
# Create the conditional kernel estimation function
def condi_kernel(data1, data2, nb_points, bandwidth):
    kernel_points1 = np.linspace(min(data1), max(data1), nb_points)
    kernel_points2 = np.linspace(min(data2), max(data2), nb_points)
    result = np.ones(len(kernel_points1)**2)
    for yi in range(0,len(kernel_points2)):
        for xi in range(0,len(kernel_points1)):
            result[xi+yi*len(kernel_points1)] = (statistics.mean(f((kernel_points1[xi] - data1) / bandwidth)*f((kernel_points2[yi] - data2) / bandwidth)) / bandwidth**2)/(statistics.mean(f((kernel_points2[yi] - data2) / bandwidth)) / bandwidth)
    return np.array([np.tile(kernel_points1, len(kernel_points1)), np.repeat(kernel_points2,len(kernel_points1)), result])

In [ ]:
# Compute the conditional kernel estimation of the returns for Credit Suisse and Microsoft
data1 = AR['Credit Suisse'].values
data2 = AR['Boeing'].values
nb_points = 50
bandwidth = np.sqrt(statistics.variance(data1))*len(data1)**(-1/5) # à trouver
l = condi_kernel(data1, data2, nb_points, bandwidth)

In [ ]:
# Plot the conditional kernel estimation of the returns for Credit Suisse and Microsoft conditioned on Credit Suisse
fig7 = go.Figure(data=[go.Scatter3d(x = l[0], y = l[1], z = l[2], mode = 'lines', marker = dict(size = 1, color = 'green'))])
fig7.update_layout(title = 'Conditional Density of Credit Suisse conditioned on Boeing', scene = dict(xaxis = dict(title = 'Credit Suisse'), yaxis = dict(title = 'Boeing'), zaxis = dict( title = 'Density'),))
fig7.show()